<a href="https://colab.research.google.com/github/amandanespoli/techchallenge3/blob/main/Fine_Tuning_Tech_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação do ambiente e importação de bibliotecas


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install bs4
!pip install tqdm
!pip install lxml

In [ ]:
!pip install unsloth

In [ ]:
!pip install --upgrade --no-deps 'unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git'

In [ ]:
!pip install --no-deps torch xformers trl peft accelerate bitsandbytes triton

In [ ]:
!pip install weave

In [ ]:
import glob
import os
from bs4 import BeautifulSoup
import json
from tqdm import tqdm
from unsloth import FastLanguageModel, is_bfloat16_supported
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
from typing import List, Dict, Any
import re
from datasets import load_dataset
import weave


# Tratamendo de Dataset - Gera arquivo JSON

Utilizacao do Dataset MedQuaAD, originado do repositorio github abaixo

In [ ]:
!git clone https://github.com/abachaa/MedQuAD

In [ ]:
# A pasta do projeto estará em /content/MedQuAD/
import os
print(os.listdir('/content/MedQuAD/'))

Extracao do coteudo xml de cada arquivo de cada diretorio para criacao de um unico arquivo jsonl no seguinte formato
* "id_origem": os.path.basename(caminho_arquivo),
* "url_origem": doc_url,
* "texto_llm": formato_llm

Sendo o conteudo do formato_llm o seguinte
* f"Responda à seguinte pergunta de forma clara e objetiva:\n[|Pergunta|]: {pergunta}[|ePergunta|]\n\n[|Resposta|]: {resposta}[|eResposta|]"

Arquivo de saida ja com limpeza e normalizacao textual ---> Remocao de espacos extras, normalizacao de quebras de linhas e padronizacao de texto



In [ ]:
DIRETORIO_XML = [
    'MedQuAD/1_CancerGov_QA/',
    'MedQuAD/2_GARD_QA/',
    'MedQuAD/3_GHR_QA/',
    'MedQuAD/4_MPlus_Health_Topics_QA/',
    'MedQuAD/5_NIDDK_QA/',
    'MedQuAD/6_NINDS_QA/',
    'MedQuAD/7_SeniorHealth_QA/',
    'MedQuAD/8_NHLBI_QA_XML',
    'MedQuAD/9_CDC_QA',
    'MedQuAD/10_MPlus_ADAM_QA',
    'MedQuAD/11_MPlusDrugs_QA',
    'MedQuAD/12_MPlusHerbsSupplements_QA'
]


dados_para_fine_tuning = []

caminhos_arquivos_xml = []

for diretorio in DIRETORIO_XML:
    caminhos_encontrados = glob.glob(os.path.join(diretorio, '*.xml'))
    caminhos_arquivos_xml.extend(caminhos_encontrados)

print(f'Número de arquivos XML encontrados: {len(caminhos_arquivos_xml)}')

for caminho_arquivo in caminhos_arquivos_xml:
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            conteudo_xml = f.read()

        soup = BeautifulSoup(conteudo_xml, 'lxml-xml')

        # pegar URL do <Document> uma vez por arquivo (atributo "url" presente no seu XML)
        doc_tag = soup.find('Document')
        doc_url = ""
        if doc_tag:
            doc_url = doc_tag.get('url') or doc_tag.get('docid') or doc_tag.get('id') or ""
            # se atributo não existir, tentar texto interno que contenha http
            if not doc_url:
                possible = doc_tag.find(lambda t: t.string and 'http' in t.string)
                if possible:
                    doc_url = possible.get_text(strip=True)

        # coletar todos os QAPair individuais (mais robusto que buscar apenas QAPairs container)
        qapairs = soup.find_all('QAPair')
        if not qapairs:
            # se não houver QAPair individuais, tentar extrair de containers <QAPairs>
            containers = soup.find_all('QAPairs')
            for c in containers:
                qapairs.extend(c.find_all('QAPair'))

        # fallback: se ainda vazio, parear <Question> com o próximo <Answer>
        if not qapairs:
            questions = soup.find_all('Question')
            if not questions:
                print(f"[DEBUG] arquivo {os.path.basename(caminho_arquivo)} sem tag <QAPair> ou <Question>, tags topo: {[t.name for t in soup.find_all()[:10]]}")
                continue
            for qtag in questions:
                atag = qtag.find_next(lambda t: t.name and t.name.lower() == 'answer')
                pergunta = ' '.join(qtag.get_text(strip=True).split())
                resposta = ' '.join(atag.get_text(strip=True).split()) if atag else ""
                if pergunta and resposta:
                    #formato_llm = f"Pergunta: {pergunta}\nResposta: {resposta}\n"
                    formato_llm = f"Responda à seguinte pergunta de forma clara e objetiva:\n[|Pergunta|]: {pergunta}[|ePergunta|]\n\n[|Resposta|]: {resposta}[|eResposta|]"
                    dados_para_fine_tuning.append({
                        "id_origem": os.path.basename(caminho_arquivo),
                        "url_origem": doc_url,
                        "texto_llm": formato_llm
                    })
            print(f" - Processado com sucesso (fallback): {os.path.basename(caminho_arquivo)}")
            continue


        for registro in qapairs:

            pergunta_tag = registro.find('Question')
            resposta_tags = registro.find_all('Answer')

            pergunta = pergunta_tag.get_text(strip=True) if pergunta_tag else ""
            resposta = ' '.join(' '.join(a.get_text(strip=True).split()) for a in resposta_tags) if resposta_tags else ""

            pergunta = ' '.join(pergunta.split())
            resposta = ' '.join(resposta.split())


            if pergunta and resposta:

                #formato_llm = f"Pergunta: {pergunta}\nResposta: {resposta}\n"
                formato_llm = f"Responda à seguinte pergunta de forma clara e objetiva:\n[|Pergunta|]: {pergunta}[|ePergunta|]\n\n[|Resposta|]: {resposta}[|eResposta|]"

                dados_para_fine_tuning.append({
                    "id_origem": os.path.basename(caminho_arquivo),
                    "url_origem": doc_url,
                    "texto_llm": formato_llm
                })


        print(f" - Processado com sucesso: {os.path.basename(caminho_arquivo)}")

    except Exception as e:
        print(f" - Erro ao processar arquivo {os.path.basename(caminho_arquivo)}: {e}")


print("\n-- Dados para fine-tuning extraídos --")
print(f"Total pares extraídos: {len(dados_para_fine_tuning)}")
for dado in dados_para_fine_tuning[:5]:  # Exibir apenas os primeiros
    print(f"Origem: {dado['id_origem']}")
    print(f"URL: {dado.get('url_origem','')}")
    print(dado['texto_llm'])
    print("-" * 20)


arquivo_saida = '/content/drive/MyDrive/Projeto Tech Challenge - Fase 3/dados_fine_tuning_medquad.jsonl'
print(f"\nEscrevendo {len(dados_para_fine_tuning)} pares Q&A no arquivo {arquivo_saida}...")

try:
    with open(arquivo_saida, 'w', encoding='utf-8') as outfile:
        for entry in tqdm(dados_para_fine_tuning, desc="Escrevendo JSONL"):
            # json.dumps() converte o dicionário em uma string JSON em uma única linha
            json_line = json.dumps(entry, ensure_ascii=False)
            outfile.write(json_line + '\n')

    print(f"\nProcessamento concluído. Dataset pronto em: {arquivo_saida}")

except Exception as e:
    print(f"ERRO ao escrever JSONL: {e}")


# Formatação de JSON gerado na seção anterior

Considerando que o arquivo de saida anterior tem o seguinte formato
* f"Responda à seguinte pergunta de forma clara e objetiva:\n[|Pergunta|]: {pergunta}[|ePergunta|]\n\n[|Resposta|]: {resposta}[|eResposta|]"

Foi necessario adaptar abaixo para que as intrucoes, perguntas e repostas ficassem separas em campos diferentes
* final_structure: Dict[str, List[str]] = {
    "instruction": [],

    "input": [],

    "output": []
}

Realizada normalizacao dos espacos e insercao no arquivo apenas se identificada existencia do par pergunta e reposta.


In [ ]:
from typing import List, Dict, Any
import json
import re

# --- Nomes dos Arquivos ---
INPUT_FILE = "/content/drive/MyDrive/Projeto Tech Challenge - Fase 3/dados_fine_tuning_medquad.jsonl"
OUTPUT_FILE = "/content/drive/MyDrive/Projeto Tech Challenge - Fase 3/dados_analise_estrutura.json" # Mudança para .json (Não é JSONL)

# --- Definição dos Marcadores e Instrução Estática ---
STATIC_INSTRUCTION = "Responda à seguinte pergunta de forma clara e objetiva:"
# marcadores legacy (caso alguma entrada use esse formato)
PROMPT_START_MARKER = "Pergunta: "
PROMPT_END_MARKER = "\nResposta:"
# marcadores usados no seu formato gerado anteriormente
MARK_Q_START = r"\[\|Pergunta\|\]:"
MARK_Q_END = r"\[\|ePergunta\|\]"
MARK_A_START = r"\[\|Resposta\|\]:"
MARK_A_END = r"\[\|eResposta\|\]"

# --- Variáveis para Estrutura Final de Listas ---
final_structure: Dict[str, List[str]] = {
    "instruction": [],
    "input": [],
    "output": []
}

print(f"Processando arquivo: {INPUT_FILE}")

try:
    with open(INPUT_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if not line.strip():
                continue

            try:
                data = json.loads(line)
            except json.JSONDecodeError:
                print("Erro ao decodificar JSON na linha (ignorado).")
                continue

            # 1) obter o texto que contém pergunta/resposta (suporta chaves diferentes)
            texto = ""
            if isinstance(data, dict):
                if "texto_llm" in data and isinstance(data["texto_llm"], str):
                    texto = data["texto_llm"]
                else:
                    # tentativas alternativas se dataset vier em formato prompt/completion
                    prompt = data.get("prompt", "")
                    completion = data.get("completion", "")
                    if prompt or completion:
                        texto = prompt + "\n" + completion
                    else:
                        # algumas versões podem usar 'prompt'/'response' ou 'question'/'answer'
                        texto = data.get("texto", "") or data.get("question", "") + "\n" + data.get("answer", "")

            if not texto:
                continue

            pergunta = ""
            resposta = ""

            # 2) tentar extrair pelo padrão [|Pergunta|] ... [|ePergunta|] e [|Resposta|] ... [|eResposta|]
            q_match = re.search(MARK_Q_START + r"\s*(.*?)\s*" + MARK_Q_END, texto, flags=re.DOTALL)
            a_match = re.search(MARK_A_START + r"\s*(.*?)\s*" + MARK_A_END, texto, flags=re.DOTALL)

            if q_match and a_match:
                pergunta = ' '.join(q_match.group(1).split())
                resposta = ' '.join(a_match.group(1).split())
            else:
                # 3) fallback para formato "Pergunta: ...\nResposta: ..."
                start_index = texto.find(PROMPT_START_MARKER)
                end_index = texto.find(PROMPT_END_MARKER)
                if start_index != -1 and end_index != -1 and end_index > start_index:
                    pergunta = texto[start_index + len(PROMPT_START_MARKER): end_index].strip()
                    # resposta: tudo após PROMPT_END_MARKER
                    resposta = texto[end_index + len(PROMPT_END_MARKER):].strip()
                else:
                    # 4) último fallback: tentar separar por duas linhas (primeira linha = pergunta, restante = resposta)
                    parts = [p for p in texto.splitlines() if p.strip()]
                    if len(parts) >= 2:
                        pergunta = parts[0].strip()
                        resposta = " ".join(p.strip() for p in parts[1:]).strip()

            # normaliza espaços
            pergunta = ' '.join(pergunta.split()) if pergunta else ""
            resposta = ' '.join(resposta.split()) if resposta else ""

            # adicionar se ambos existirem
            if pergunta and resposta:
                final_structure["instruction"].append(STATIC_INSTRUCTION)
                final_structure["input"].append(f"question:{pergunta}")
                final_structure["output"].append(f"answer:{resposta}")

except FileNotFoundError:
    print(f"Erro: O arquivo de entrada '{INPUT_FILE}' não foi encontrado.")

print(f"\nTotal de pares extraídos e formatados: {len(final_structure['instruction'])}")

# --- 4. Geração do Único Arquivo JSON ---
if final_structure["instruction"]:
    try:
        with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
            # Usa json.dump para escrever o objeto completo em uma única estrutura
            json.dump(final_structure, f, indent=4, ensure_ascii=False)

        print(f"✅ Arquivo de saída gerado com sucesso: '{OUTPUT_FILE}'")


    except Exception as e:
        print(f"Erro ao escrever o arquivo de saída: {e}")
else:
    print("❌ Nenhuma linha válida foi processada. O arquivo de saída não foi gerado.")

# Treinamento do modelo com entrada do JSON formatado

Preparacao do modelo para fine-tuning eficiente com
* Ativacao da quantizacao em 4 bits para reducao no uso de memoria e carregamento do modelo llm LLaMA 3 8B com otimizacao do Unsloth.

In [ ]:
max_seq_length = 2048
dtype = torch.bfloat16 if is_bfloat16_supported() else torch.float16
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


Aplicacao PEFT usando LoRA no modelo 4 bits carregado anteriormente.

Com o objetivo de reduzir o custo de treino, os pesos originais do modelo nao sao treinados e pequenas matrizes LoRA sao adiconadas.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

* Definicao do template instrucional que define o que eh instrucao, entrada e saida esperada.

* Utilizacao do Token de fim de sequencia (EOS) que indica o fim da resposta.

* Definicao do texto final de treino com criacao da coluna text, dataset preparado para tokenizacao e fine-tuning

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context

### Instruction:
{}

### Input:
{}

### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
  instructions = examples["instruction"]
  inputs       = examples["input"]
  outputs      = examples["output"]
  texts = []
  for instruction, input, output in zip(instructions, inputs, outputs):

    text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
    texts.append(text)
  return {"text": texts, }
pass

OUTPUT_FILE = "/content/drive/MyDrive/Projeto Tech Challenge - Fase 3/dados_analise_estrutura.json"

dataset = load_dataset("json", data_files=OUTPUT_FILE, split="train")
dataset = dataset.map(formatting_prompts_func, batched=True,)


Configuracao do treinador de fine-tuning supervisonado para o LLM quantizado e adaptado com LoRA, definindo parametros eficientes para treinar um modelo grande de forma eficiente.

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Gatilho que inicia efetivamente o fine-tuning do modelo e retorna estatisticas do treinamento.

In [ ]:
trainer_stats = trainer.train()

Preparacao do modelo para inferencia com a construcao de um prompt no formato Alpaca.

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Responda à seguinte pergunta de forma clara e objetiva:",
        "What is the relationship between Noonan syndrome and polycystic renal disease?",
        "",
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Mesmo comando de inferencia, mas com utilizacao do TextStreamer que permite visualizar o texto a medido que o modelo o produz.

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Responda à seguinte pergunta de forma clara e objetiva:",
        "What is the relationship between Noonan syndrome and polycystic renal disease?",
        "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


Salvamento do modelo LoRA treinado e seu tokenizer.

In [ ]:
model.save_pretrained("/content/drive/MyDrive/Projeto Tech Challenge - Fase 3/lora_model")
tokenizer.save_pretrained("/content/drive/MyDrive/Projeto Tech Challenge - Fase 3/lora_model")


Login no Hugging face

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Exportacao do modelo treinado no formato GGUF, enviando o modelo para o repositorio Hugging Face.

In [ ]:
model.push_to_hub_gguf('llama-3-8b-bnb-4bit-perguntas-respostas-medicina', tokenizer, quantization_method = 'q4_k_m')